In [1]:
%load_ext autoreload
%autoreload 2
import numpy as np
import tensorflow as tf
from processing_funcs import *
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import Input, Conv1D, MaxPooling1D, UpSampling1D
from tensorflow.python.keras.regularizers import l1

2024-11-21 17:15:32.493345: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-21 17:15:40.365287: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-21 17:15:40.365358: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-21 17:15:41.085008: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-21 17:15:56.652271: W tensorflow/stream_executor/pla

In [2]:
# Load dataframe and prepare samples
df = load_df()
df = process_wfs(df)


Loading York Data 1
Combining into one Dataframe!
Processing wf 1/83
Processing wf 2/83
Processing wf 3/83
Processing wf 4/83
Processing wf 5/83
Processing wf 6/83
Processing wf 7/83
Processing wf 8/83
Processing wf 9/83
Processing wf 10/83
Processing wf 11/83
Processing wf 12/83
Processing wf 13/83
Processing wf 14/83
Processing wf 15/83
Processing wf 16/83
Processing wf 17/83
Processing wf 18/83
Processing wf 19/83
Processing wf 20/83
Processing wf 21/83
Processing wf 22/83
Processing wf 23/83
Processing wf 24/83
Processing wf 25/83
Processing wf 26/83
Processing wf 27/83
Processing wf 28/83
Processing wf 29/83
Processing wf 30/83
Processing wf 31/83
Processing wf 32/83
Processing wf 33/83
Processing wf 34/83
Processing wf 35/83
Processing wf 36/83
Processing wf 37/83
Processing wf 38/83
Processing wf 39/83
Processing wf 40/83
Processing wf 41/83
Processing wf 42/83
Processing wf 43/83
Processing wf 44/83
Processing wf 45/83
Processing wf 46/83
Processing wf 47/83
Processing wf 48/83

In [ ]:
# Configuration variables
HLactivation = 'relu'               # Activation for hidden layers
outputactivation = 'sigmoid'        # Activation for output layer
filtersize = 5                      # Variable kernel size for Conv1D layers
use_l1_regularization = True        # Toggle L1 regularization on or off
l1_lambda = 0.001                   # L1 regularization coefficient, if applicable

# Loss weights
weighted_loss_type = 'exponential'  # Options: 'exponential' or 'square'
weight_min = 100                  # Minimal freq_bin index to weight. Avoids weighting the near zero "peak".

# Channel numbers for each Conv1D layer
channel_num1 = 32
channel_num2 = 16
channel_num3 = 8

# Max pooling down/upsampling ratio
MP_ratio = 4

# Define a weighted loss function based on the variable choice
def weighted_mse(y_true, y_pred):
    if weighted_loss_type == 'exponential':
        weights = tf.exp(y_true)
    elif weighted_loss_type == 'square':
        weights = tf.square(y_true)
    else:
        raise ValueError("Invalid weighted_loss_type. Choose 'exponential' or 'square'.")
    # If we're below the cutoff, just replace the weight with the lowest calculated weight.
    # This way we don't care much about the universal near zero "peak"
    weights[0:weight_min] = np.min(weights)
    return tf.reduce_mean(weights * tf.square(y_true - y_pred))

# Define the encoder
input_data = Input(shape=(1000, 1))

# Optional L1 regularization
if use_l1_regularization:
    regularizer = l1(l1_lambda)
else:
    regularizer = None

# Encoder with variable channel numbers
x = Conv1D(channel_num1, filtersize, activation=HLactivation, padding='same', kernel_regularizer=regularizer)(input_data)
x = MaxPooling1D(MP_ratio, padding='same')(x)
x = Conv1D(channel_num2, filtersize, activation=HLactivation, padding='same', kernel_regularizer=regularizer)(x)
x = MaxPooling1D(MP_ratio, padding='same')(x)
x = Conv1D(channel_num3, filtersize, activation=HLactivation, padding='same', kernel_regularizer=regularizer)(x)
encoded = MaxPooling1D(MP_ratio, padding='same')(x)

# Decoder with variable channel numbers
x = Conv1D(channel_num3, filtersize, activation=HLactivation, padding='same', kernel_regularizer=regularizer)(encoded)
x = UpSampling1D(MP_ratio)(x)
x = Conv1D(channel_num2, filtersize, activation=HLactivation, padding='same', kernel_regularizer=regularizer)(x)
x = UpSampling1D(MP_ratio)(x)
x = Conv1D(channel_num1, filtersize, activation=HLactivation, padding='same', kernel_regularizer=regularizer)(x)
x = UpSampling1D(MP_ratio)(x)
decoded = Conv1D(1, filtersize, activation=outputactivation, padding='same')(x)

# Define the autoencoder model
autoencoder = Model(input_data, decoded)

# Compile the model using the weighted loss function
autoencoder.compile(optimizer='adam', loss=weighted_mse)

# Print a summary of the model to verify the architecture
autoencoder.summary()


In [ ]:
# Load the data
